# Example of FaIRv2.1

FaIR v2.1 is object-oriented and designed to be more flexible than its predecessors. This does mean that setting up a problem is different to before - gone are the days of 60 keyword arguments to `fair_scm` and we now use hierarchical classes with fewer arguments that in the long run should be easier to use. Of course, there is a learning curve, and will take some getting used to. This tutorial aims to walk through a simple problem using FaIR 2.1.

The structure of FaIR 2.1 centres around the `FAIR` class, which contains all information about the scenario(s), the forcer(s) we want to investigate, and any configurations specific to each species and the response of the climate.

A run is defined as follows:

```
fair = FAIR(scenarios, configs, timestep, run_config)
```

This loads scenarios and configurations into FaIR. The model is run with

```
fair.run()
```

results are stored within the `fair` instance, and can be obtained such as

```
print(fair.temperature)
```

Multiple `scenarios` and `configs` can be supplied in a `FAIR` instance, and due to internal parallelisation is the fastest way to run the model (100 ensemble members per second for 1750-2100 on my Mac for an emissions driven run). The total number of scenarios that will be run is the product of `scenarios` and `configs`. For example we might want to run three emissions `scenarios` -- let's say SSP1-2.6, SSP2-4.5 and SSP3-7.0 -- using climate calibrations (`configs`) from the UKESM, GFDL, MIROC and NorESM climate models. This would give us a total of 3$\times$4 = 12 ensemble members in total which are run in parallel.

The most difficult part to learning FaIR 2.1 is correctly defining the `scenarios` and `configs`. As in previous versions of FaIR, there is a lot of flexibility, and simplifying the calling interface (`fair_scm` in v1.x) has come at the cost of most of these options being moved into one of the `SpeciesID`, `ClimateResponse`, `SpeciesConfig` or `RunConfig` instances.

## Recommended order for setting up a problem

In this tutorial the recommended order in which to define a problem is set out step by step, and is as follows:

1. Define each `SpeciesID`: what is forcing our model and how?
2. Assign data (emissions, concentration, forcing etc.) to each `SpeciesID` in the `Species` class
3. Create one or more `Scenario`s, which are each defined as a list of `Species`. The list of `Scenario`s is the first attribute of the `FAIR` class above.
4. Create instances of `ClimateResponse`: how we want the climate to behave for each `Scenario`.
5. For each of the `ClimateResponse` instances, create a list of `SpeciesConfig`s that cover the same `SpeciesID`s that go into the `Scenario`s (in the same order).
6. Create a list of `Config`s containing lists of `ClimateResponse` and `SpeciesConfig`. This goes into the second attribute of the `FAIR` class.
7. Define a `timestep` value in years. This is the third attribute of `FAIR`. It's optional if the timestep is one year.
8. Define any additional top-level run configurations in the `RunConfig` class. This is the fourth attribute of `FAIR`. Again if using default run configurations, it's optional.
9. Run `FAIR`.

## Defining the scope of our problem

### The SpeciesID

The starting point is the `SpeciesID` class which defines the forcers -- anthropogenic or natural -- that are present in your scenario. Each `SpeciesID` is assigned a name that is used to distinguish it from other species, along with a `Category` class that defines what the thing actually is, and a `RunMode` that defines how we want to include this particular `SpeciesID` in the model.

In [ ]:
from fair21 import SpeciesID, Category, RunMode

In [ ]:
print(SpeciesID.__doc__)

In this example we'll start off running a scenario with CO2 from fossil fuels and industry, CO2 from AFOLU, CH4, N2O, and Sulfur (note you don't need the full 40 species used in v1.1-1.6). To highlight some of the functionality we'll run CO2 and Sulfur emissions-driven, and CH4 and N2O concentration-driven. (This is akin to an `esm-ssp585` kind of run from CMIP6, though with fewer species). We'll use totally fake data here - this is not intended to represent a real-world scenario but just to highlight how FaIR works.

First, we need to define the `SpeciesID`s of all of the things we want to include. Full simulations may have 50 or more species included, so it can be beneficial to build a `dict` of `SpeciesID`s, particularly as we will re-use them later. This is what we'll do here.

Finally, it's important to be consistent about the `name` attribute given to each `SpeciesID` as this will be used later in the `SpeciesConfig`s.

In [ ]:
species_ids = {}
species_ids['co2_ffi'] = SpeciesID('CO2 Fossil fuel and industrial', Category.CO2_FFI, RunMode.EMISSIONS)
species_ids['co2_afolu'] = SpeciesID('CO2 AFOLU', Category.CO2_AFOLU, RunMode.EMISSIONS)
species_ids['co2'] = SpeciesID('CO2', Category.CO2, RunMode.FROM_OTHER_SPECIES)
species_ids['ch4'] = SpeciesID('CH4', Category.CH4, RunMode.CONCENTRATION)
species_ids['n2o'] = SpeciesID('N2O', Category.N2O, RunMode.CONCENTRATION)
species_ids['sulfur'] = SpeciesID('Sulfur', Category.SULFUR, RunMode.EMISSIONS)
species_ids['aci'] = SpeciesID('Aerosol-cloud interactions', Category.AEROSOL_CLOUD_INTERACTIONS, RunMode.FROM_OTHER_SPECIES)

There's a few things going on here. We see that `Category` and `RunMode` have specific values that they can take: they are `dataclass`es (a new feature in Python 3.7, so FaIR 2.1 requires at least this version of Python). Let's explore these a litte more closely.

### Category class

We see there's 20 or valid values for `Category` which can be attached to a `SpeciesID` or `Species`, and everything included in FaIR must fall into one of these categories:

In [ ]:
dir(Category)

Trying to access a `Category` outside of this raises an error. But miscellaneous forcings that are not part of the above categorization can be included as`Category.OTHER` (cf. FaIR v1.0 considering non-CO2 forcings).

In [ ]:
try:
    Category.COSMIC_RAYS
except Exception as exc:
    print(f"{str(exc)} is not a valid Category")

### RunMode class

The `RunMode` defines how we want to drive the model for each `Species`. It is defined as a attribute of `SpeciesID` and takes one of four possible values:

1. `RunMode.EMISSIONS`
2. `RunMode.CONCENTRATION`
3. `RunMode.FORCING`
4. `RunMode.FROM_OTHER_SPECIES`

The first three are self-explanatory. `RunMode.FROM_OTHER_SPECIES` is used when there's no direct emission or concentration relationship to a climate forcing agent.

In our example above, we use `RunMode.FROM_OTHER_SPECIES` for CO2 because the total burden of atmospheric CO2 comes from two sources: fossil fuel + industry and AFOLU (which are emissions-driven). In reality this is true for many emissions sources, but CO2 AFOLU has a special treatment in FaIR and as such when running emissions-driven it is necessary to separate the sources.

Not all `RunMode`s are defined or sensible for all `Species`.

In [ ]:
try:
    SpeciesID('Solar', Category.SOLAR, RunMode.EMISSIONS)
except Exception as exc:
    print(str(exc))

and in fact, we can't run just the plain `Category.CO2` mode emissions-driven either.

In [ ]:
try:
    SpeciesID('Total CO2', Category.CO2, RunMode.EMISSIONS)
except Exception as exc:
    print(str(exc))

That's not to say that `Category.CO2` can't take emissions: when we provide a concentration-driven CO2 run, we have no idea what the source of the CO2 actually was so we have to assign it all to one group. We'll revisit this when we come to concentration-driven CO2 runs.

The other thing we have done in our example is defined a `Category.AEROSOL_CLOUD_INTERACTIONS` that calculated this forcing from sulfur emissions.

## Assigning data: the Species class

`SpeciesID` just tells us what is present. To build a `Scenario`, we have to populate it with `Species` that assigns emissions, concentrations or forcing to a `SpeciesID`. We can run multiple `Scenario`s per evaluation of FaIR, and therefore we can have several instances of `Species` pointing to the same `SpeciesID` when running a scenario batch. This is what we will do below, running two scenarios.

In [ ]:
from fair21 import Species
import numpy as np

In [ ]:
print(Species.__doc__)

Here, we'll run two 50 year long scenarios. The first will have emissions and concentrations for the 50 years that roughly reflect present-day emissions or concentrations to each `Species`. The second will ramp up to that level in year 50 from pre-industrial levels.

Again, we'll use `dict`s to group together `Scenario`s. We require a `list` of `Species` for each `Scenario`. The order in which `Species` are input doesn't matter (unlike in FaIR 1.1-1.6), but it must be consistent across `Scenario`s and later on in `SpeciesConfig`s.

Note also that we need to define the total CO2 and Aerosol-cloud interaction `Species`, but we do not assign emissions or forcing to them.

For CH4 and N2O (and in fact, every greenhouse gas), the pre-industrial concentrations are stored as `baseline_concentrations` in a module of species defaults, and can be imported.

In [ ]:
from fair21.defaults.default_species_config import default_species_config

ch4_concentration_pi = default_species_config['ch4'].baseline_concentration
n2o_concentration_pi = default_species_config['n2o'].baseline_concentration
print(f"Default baseline concentrations: CH4 = {ch4_concentration_pi} ppb, N2O = {n2o_concentration_pi} ppb")

# for a list of defined defaults
print()
print("Default species defined:", list(default_species_config.keys()))

In [ ]:
species = {}

# remember the order!
species['abrupt'] = [
    Species(species_ids['co2_ffi'], emissions=np.ones(50)*38), # default unit GtCO2/yr
    Species(species_ids['co2_afolu'], emissions=np.ones(50)*3) ,
    Species(species_ids['ch4'], concentration=np.ones(50)*1800), # default unit ppb
    Species(species_ids['n2o'], concentration=np.ones(50)*325),
    Species(species_ids['sulfur'], emissions=np.ones(50)*100), # default unit MtSO2/yr
    Species(species_ids['co2']),
    Species(species_ids['aci'])
]

species['ramp'] = [
    Species(species_ids['co2_ffi'], emissions=np.linspace(0, 38, 50)), # default unit GtCO2/yr
    Species(species_ids['co2_afolu'], emissions=np.linspace(0, 3, 50)), 
    Species(species_ids['ch4'], concentration=np.linspace(ch4_concentration_pi, 1800, 50)),
    Species(species_ids['n2o'], concentration=np.linspace(n2o_concentration_pi, 325, 50)),
    Species(species_ids['sulfur'], emissions=np.ones(50)*100), # default unit MtSO2/yr
    Species(species_ids['co2']),
    Species(species_ids['aci'])
]

We see that `SpeciesID`s are re-used as they uniquely define what each substance actually is, but `Species` are defined once per `Scenario` as we can investigate different forcing pathways with the same setup.

Let's see an example of what the `Species` class looks like to the code.

In [ ]:
species['ramp'][3]

## Defining Scenarios

We have declared all of our `Species`, and we now need to formally assign them to `Scenario`s in the code.

In [ ]:
from fair21 import Scenario

In [ ]:
print(Scenario.__doc__)

`scenarios` go into `fair` as a list, and we can take a peek at what our `Scenario` actually looks like to FaIR.

In [ ]:
scenarios = []
for scenario in ['abrupt', 'ramp']:
    scenarios.append(Scenario(scenario, species[scenario]))

## The Configs

`Scenario`s define what goes into the model, but `Config`s define the model behaviour. There are two types of configs (well, actually three if you include the overall run configs, but these are defined at the top level and we'll revisit these later):

1. `ClimateResponse` defines the climate sensitivity, ocean heat capacities, heat transfer coefficient, etc. It defines how forcing is converted to temperature in the model. Only one `ClimateResponse` per config should be defined.
2. `SpeciesConfig` defines things such as greenhouse gas lifetimes, radiative forcing efficacy, gas cycle airborne fraction, and many other things. It has a lot of options, and a lot of flexibility. A list containing a `SpeciesConfig` for each defined species in the scenarios needs to be provided.

`Config`s are defined as follows:

```
config = Config(name, climate_response, species_configs)
```

We then provide a list of configs to FaIR.

In this example, we'll provide three configs representing high, medium and low climate sensitivity climate models, and also vary the `SpeciesConfig` in each to provide some variation in the carbon cycle feedbacks and the strength of the aerosol forcing.

### Climate response

We'll start with the climate response which is slightly easier. This is an $n$-box energy balance model, where $n=3$ by default (this can be changed in the `RunConfig`). 

In [ ]:
from fair21 import ClimateResponse

In [ ]:
print(ClimateResponse.__doc__)

In [ ]:
climate_response = {}
climate_response['high'] = ClimateResponse(
    ocean_heat_capacity = np.array([5, 15, 80]),
    ocean_heat_transfer = np.array([0.6, 1.3, 1.0]),
    deep_ocean_efficacy = 1.29,
    stochastic_run = False
)
climate_response['central'] = ClimateResponse(
    ocean_heat_capacity = np.array([8, 14, 100]),
    ocean_heat_transfer = np.array([1.1, 1.6, 0.9]),
    deep_ocean_efficacy = 1.1,
    stochastic_run = False
)
climate_response['low'] = ClimateResponse(
    ocean_heat_capacity = np.array([6, 11, 75]),
    ocean_heat_transfer = np.array([1.7, 2.0, 1.1]),
    deep_ocean_efficacy = 0.8,
    stochastic_run = False
)

### Species configuration

There are a lot of options - but only a few per species are used. And to make things easier, the `default_species_config` module contains many really useful defaults.

In [ ]:
from fair21 import SpeciesConfig

In [ ]:
print(SpeciesConfig.__doc__)

Here's an example of a default:

In [ ]:
default_species_config['co2']

Again let's create a `dict` with the key `config` name from the `ClimateResponse` and a list of `SpeciesConfig`s.

In most cases you won't need to modify the defaults too much and they can be used as a starting point. So let's do this, looping through `config` and `species` but just for fun maybe also vary some of the sensivities in the carbon cycle, and switch the aerosol indirect treatment to Stevens.

Do remember, that the order of the list elements corresponding to each `Species` must match the order they appear in the `Scenario`.

#### Fill in defaults

In [ ]:
species_ids

<div class="alert alert-block alert-warning">
It's really important to use `copy` here, otherwise we are just creating multiple references to the default dict.
</div>

In [ ]:
import copy
species_configs = {}

for config_label in ['high', 'central', 'low']:
    species_configs[config_label] = []
    for species_label in ['co2_ffi', 'co2_afolu', 'ch4', 'n2o', 'sulfur', 'co2', 'aci']:
        # use copy here!
        species_configs[config_label].append(copy.copy(default_species_config[species_label]))
        # the defaults also contain the SpeciesID that has a default name and run mode. We want to override this.
        species_configs[config_label][-1].species_id = species_ids[species_label]

#### Modify carbon cycle, sulfur and ERFaci configs

In small problems like this you might be able to remember the indices of each specie, but in case you need a reminder - and this is very useful for big problems such as a full CMIP6 run:

In [ ]:
for ispecconf in range(len(species_configs[config_label])):
    print (ispecconf, species_configs[config_label][ispecconf].species_id.name)

So if we want to modify the carbon cycle, we have to modify the properties in `SpeciesConfig` of CO2, which is list element number 5.

First, let's examine the defaults:

In [ ]:
print(
    species_configs['central'][5].iirf_airborne,
    species_configs['central'][5].iirf_cumulative,
    species_configs['central'][5].iirf_temperature
)

Again, it's fake data, but let's double the feedback parameters in the "high" case and set them to zero in the "low" case.

In [ ]:
species_configs['high'][5].iirf_airborne = species_configs['central'][5].iirf_airborne*2 
species_configs['high'][5].iirf_cumulative = species_configs['central'][5].iirf_cumulative*2 
species_configs['high'][5].iirf_temperature = species_configs['central'][5].iirf_temperature*2

species_configs['low'][5].iirf_airborne = 0
species_configs['low'][5].iirf_cumulative = 0
species_configs['low'][5].iirf_temperature = 0

For Sulfur, we'll override the baseline (pre-industrial) emissions from the default value provided, which is 1750.

From above, we know Sulfur emissions is array index 4, and the aerosol-cloud interactions forcing is array index 6.

We'll use the beta (scale) and Qn (Sulfur) from Stevens (2015). Later we also need to change the aerosol-cloud interactions `RunMode` in the `RunConfig` settings too, to set this to the Stevens (2015) treatment rather than the FaIR v1.3 default.

In [ ]:
for config_label in ['high', 'central', 'low']:
    species_configs[config_label][4].baseline_emissions = 0
    species_configs[config_label][6].aci_params = {
        "scale": 1, "Sulfur": 60
    }

### now, let's assign to Configs

In [ ]:
from fair21 import Config

configs = []
for config_label in ['high', 'central', 'low']:
    configs.append(Config(config_label, climate_response[config_label], species_configs[config_label]))

## modify the RunConfig

change the aerosol forcing behaviour to Stevens (2015)

In [ ]:
from fair21 import RunConfig, ACIMethod

In [ ]:
run_config = RunConfig(aci_method=ACIMethod.STEVENS2015)

In [ ]:
run_config

## Now, we can run FaIR

In [ ]:
from fair21 import FAIR

In [ ]:
timestep = 1  # not required, but demonstrated
fair = FAIR(scenarios, configs, timestep, run_config)

In [ ]:
fair.run()

In [ ]:
fair.temperature.shape

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
pl.plot(np.arange(50), fair.temperature[:,0,:,0,0], label=['high','medium','low'])
pl.title('Ramp scenario: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')
pl.legend()

In [ ]:
pl.plot(np.arange(50), fair.concentration_array[:,1,:,5,0], label=['high','medium','low'])
pl.title('Ramp scenario: CO2')
pl.xlabel('year')
pl.ylabel('CO2 (ppm)')
pl.legend()

In [ ]:
fair.forcing_sum_array.shape

In [ ]:
pl.plot(np.arange(50), fair.forcing_sum_array[:,0,:,0,0], label=['high','medium','low'])
pl.title('Ramp scenario: forcing')
pl.xlabel('year')
pl.ylabel('Total ERF (W m$^{-2}$)')
pl.legend()

In [ ]:
pl.plot(np.arange(50), fair.temperature[:,1,:,0,0], label=['high','medium','low'])
pl.title('Abrupt scenario: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')
pl.legend()

In [ ]:
pl.plot(np.arange(50), fair.forcing_sum_array[:,1,:,0,0], label=['high','medium','low'])
pl.title('Abrupt scenario: forcing')
pl.xlabel('year')
pl.ylabel('Total ERF (W m$^{-2}$)')
pl.legend()

In [ ]:
pl.plot(np.arange(50), fair.concentration_array[:,0,:,5,0], label=['high','medium','low'])
pl.title('Abrupt scenario: CO2')
pl.xlabel('year')
pl.ylabel('CO2 (ppm)')
pl.legend()